# Chat Bot
***
## Table of Contents
***

## 1. Introduction


## 2. Environmental Variables
The API keys and environmental variables should never be hardcoded or exposed publicly. The [python-dotenv](https://pypi.org/project/python-dotenv/) library facilitates secure access to variables defined in a `.env` file.

In [ ]:
import os
from getpass import getpass

try:
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    raise ImportError("Error: 'python-dotenv not installed")